In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
data_train_path = 'Fruits_Vegetables/train'
data_train_test = 'Fruits_Vegetables/test'
data_train_val = 'Fruits_Vegetables/validation'